---
title: LiDAR
tags: [geo, geospatial,]
keywords: geo-python gdal QGIS
summary: "Digest lidar data in Python"
sidebar: mydoc_sidebar
permalink: lidar.html
folder: geopy
---


{% include tip.html content="Use [*QGIS*](geo_software.html#qgis) to display geospatial data products." %}

## Laspy

* [Documentation](https://laspy.readthedocs.io/en/latest/)
* [Tutorials](https://laspy.readthedocs.io/en/latest/tut_background.html)

{% include windows.html content="In order to work with the *LAStools* plugin in *QGIS*, download `LAStools.zip` from [https://www.cs.unc.edu/~isenburg/lastools/](https://www.cs.unc.edu/~isenburg/lastools/) (not a freeware), and unpack the zip folder to `C:\LAStools\`. Make sure that the directory `C:\LAStools\bin` exists and contains `las2shp.exe`." %}


### Install

Type in *Anaconda Prompt*:

```
conda install -c conda-forge laspy
```

Find advanced installation instructions on [laspy.readthedocs.io](https://laspy.readthedocs.io/en/latest/tut_part_1.html).

### Usage
*laspy* uses *numpy* arrays to store data and this is why both libraries need be imported to read a *las* file:

In [1]:
import numpy
import laspy
las_file_name = os.path.abspath("") + "/data/subsample.las"

Then, we can load a *las* file with `file_object = laspy.file.File(`file_name`, mode="rw")`. Allowed `mode`s are `"r"` (read), `"w"` (write), and `"rw"` (read-write).

To read essential data (points with attributes) from a *las* file, extract the points (*numpy* array) and have a look at the *dtypes*. The following code block uses a `with` statement to avoid that the *las* file is locked by *Python*:

In [2]:
with laspy.file.File(las_file_name, mode="r") as las_file:
    pts = las_file.points

print(pts.dtype)

[('point', [('X', '<i4'), ('Y', '<i4'), ('Z', '<i4'), ('intensity', '<u2'), ('flag_byte', 'u1'), ('classification_flags', 'u1'), ('classification_byte', 'u1'), ('user_data', 'u1'), ('scan_angle', '<i2'), ('pt_src_id', '<u2'), ('gps_time', '<f8')])]


`('X', '<i4'), ('Y', '<i4'), ('Z', '<i4')` tells us, that the first three array entries are a point's X, Y, and Z coordinates, , followed by `'intensity'` and so on. So the first row of points looks like this:

The *las* file has many more attribute such as color ranges (*Red*, *Green*, *Blue*), spatial reference, or *NIR* (near-infrared). The following code block extracts and prints some of the file properties including its header:

In [10]:
with laspy.file.File(las_file_name, mode="r") as las_file:
    print(dir(las_file))
    headers = [str(spec.name) for spec in las_file.header.header_format]
print(", ".join(headers))

['Blue', 'Classification', 'Green', 'Intensity', 'Key_Point', 'Raw_Classification', 'Red', 'Synthetic', 'Withheld', 'X', 'Y', 'Z', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_evlrs', '_header', '_mode', '_reader', '_vlrs', 'addProperty', 'assertWriteMode', 'blue', 'byte_offset_to_waveform_data', 'classification', 'classification_flags', 'close', 'define_new_dimension', 'doc', 'edge_flight_line', 'extra_bytes', 'filename', 'flag_byte', 'get_blue', 'get_byte_offset_to_waveform_data', 'get_classification', 'get_classification_flags', 'get_edge_flight_line', 'get_extra_bytes', 'get_flag_byte', 'get_gps_time', 'get_green', 'get_head

To access geospatial and point properties

In [4]:
with laspy.file.File(las_file_name, mode="rw") as las_file:
    pts = las_file.points
    x_dim = las_file.X
    y_dim = las_file.Y
    scale = las_file.header.scale[0]
    offset = las_file.header.offset[0]

    print("x_dim: " + str(x_dim))
    print("y_dim: " + str(y_dim))
    print("scale: " + str(scale))
    print("offset: " + str(offset))


x_dim: [252392 252726 252645 ... 215816 216018 216753]
y_dim: [-451699 -448771 -448837 ... -423009 -424070 -424156]
scale: 0.001
offset: 751251.0


In [9]:
import copy

las_file = laspy.file.File(las_file_name, mode="r")


print(dir(las_file.header.vlrs))

for spec in las_file.reader.point_format:
    in_spec = las_file.reader.get_dimension(spec.name)
    print(in_spec)
    print(spec.name)



['__add__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']
[252392 252726 252645 ... 215816 216018 216753]
X
[-451699 -448771 -448837 ... -423009 -424070 -424156]
Y
[502275 393328 393357 ... 407968 404934 405072]
Z
[ 146 1402 1340 ... 1967 1232 1040]
intensity
[33 33 33 ... 17 33 66]
flag_byte
[0 0 0 ... 0 0 0]
classification_flags
[0 0 0 ... 0 0 0]
classification_byte
[0 0 0 ... 0 0 0]
user_data
[0 0 0 ... 0 0 0]
scan_angle
[101 101 101 ...  89  89  89]
pt_src_id
[470157.6591803  470159.140